In [3]:
import numpy as np
import cv2
import tensorflow as tf
import pandas as pd

In [4]:
face_detection = cv2.CascadeClassifier('/Users/zeynep/Desktop/ML/facial-expression-recognition-master/haar_cascade_face_detection.xml')

camera = cv2.VideoCapture("/Users/zeynep/Desktop/powerIsPower.mp4")
#camera = cv2.VideoCapture("/Users/zeynep/Desktop/ML/facial-expression-recognition-master/imageProcessing_videos/deprem1.mp4")
camera.set(cv2.CAP_PROP_FRAME_WIDTH, 1024)
camera.set(cv2.CAP_PROP_FRAME_HEIGHT, 768)
settings = {
    'scaleFactor': 1.3,
    'minNeighbors': 5,
    'minSize': (50, 50)
}

labels = ['Surprise', 'Neutral', 'Anger', 'Happy', 'Sad']

model = tf.keras.models.load_model('/Users/zeynep/Desktop/ML/facial-expression-recognition-master/network-5Labels.h5')

output_data = []

frame=0

while True:
    ret, img = camera.read()

    if not ret:
        break

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    detected = face_detection.detectMultiScale(gray, **settings)

    for x, y, w, h in detected:
        cv2.rectangle(img, (x, y), (x + w, y + h), (245, 135, 66), 2)
        cv2.rectangle(img, (x, y), (x + w // 3, y + 20), (245, 135, 66), -1)
        face = gray[y + 5:y + h - 5, x + 20:x + w - 20]
        face = cv2.resize(face, (48, 48))
        face = face / 255.0
        sentiment_score = model.predict(np.array([face.reshape((48, 48, 1))]))
        sentiment_label = labels[sentiment_score.argmax()]
        sentiment_score = sentiment_score[0][sentiment_score.argmax()]
        output_data.append((sentiment_label, sentiment_score))
        state = sentiment_label
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, state, (x + 10, y + 15), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        frame = frame+1

    cv2.imshow('Facial Expression', img)

    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()

output_array = np.array(output_data)
#print(output_array)

# Create a dictionary to hold the data
data = {'#Frame': np.arange(1, len(output_array) + 1),
        'Sentiment Label': output_array[:, 0],
        'Sentiment Score': output_array[:, 1]
}

# Convert the dictionary to a Pandas DataFrame
df = pd.DataFrame(data)

df.to_csv('output.csv', index=False)
print(df)

[ WARN:0@634.995] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_d9lyif19nl/croot/opencv-suite_1676472756314/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@634.995] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_d9lyif19nl/croot/opencv-suite_1676472756314/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin1 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@634.995] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_d9lyif19nl/croot/opencv-suite_1676472756314/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@634.995] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_d9lyif19nl/croot/opencv-suite_1676472756314/work/modules/video

1/1 [==============================] - 0s 13ms/step
      #Frame Sentiment Label Sentiment Score
0          1           Anger      0.39381626
1          2             Sad      0.37413457
2          3             Sad      0.38107678
3          4           Anger      0.42014262
4          5           Anger      0.36908752
...      ...             ...             ...
2330    2331           Anger       0.5956368
2331    2332           Anger      0.57532704
2332    2333           Anger       0.5616084
2333    2334           Anger       0.6593909
2334    2335           Anger      0.59524804

[2335 rows x 3 columns]
